# Jupyter Notebook: Player Tracking and Re-Identification in a Single Video Feed

## Overview


This notebook implements a player tracking and re-identification pipeline for a 15-second video clip. The goal is to detect players using a fine-tuned YOLOv11 model, track them using the Norfair library, and maintain consistent player identities across frames, even when players exit and re-enter the frame. The pipeline includes:

* Object Detection: Using YOLOv11 to detect players.
* Tracking: Using Norfair to assign tracking IDs.
* Re-Identification: Using histogram-based appearance matching to maintain consistent IDs.
* Visualization: Saving tracked and re-identified outputs as videos and logging ID switches.
* Analysis: Plotting histogram similarity scores and logging ID changes.

# Installation

In [ ]:
!pip install ultralytics opencv-python-headless norfair matplotlib pandas --quiet

# Imports

In [ ]:
import cv2
import numpy as np
import random
from collections import defaultdict
from ultralytics import YOLO
from norfair import Tracker, Detection
import matplotlib.pyplot as plt
import pandas as pd

# Configuration

In [ ]:
# Video paths
video_path = '/kaggle/input/15-sec-video/15_sec_video.mp4'
reid_output_path = '/kaggle/working/reid_output.mp4'
track_output_path = '/kaggle/working/tracked_output_with_head.mp4'

# Model path
model_path = '/kaggle/input/best/pytorch/default/1/best.pt'

# Tracker parameters
distance_threshold = 150
initialization_delay = 1
num_reference_frames = 50  # Frames to build reference gallery
match_threshold = 0.85  # Histogram similarity threshold for re-identification

# Load Model and Video

In [ ]:
# Load YOLOv11 model
model = YOLO(model_path)
print("Model class names:", model.names)

# Load video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit(1)

# Video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video properties: FPS={fps}, Width={width}, Height={height}")

# Initialize Video Writers

In [ ]:
reid_out = cv2.VideoWriter(reid_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
track_out = cv2.VideoWriter(track_output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
if not reid_out.isOpened() or not track_out.isOpened():
    print("Error: Could not initialize video writer.")
    cap.release()
    exit(1)

# Tracker Setup

In [ ]:
def euclidean_distance(detection, tracked_object):
    detection_center = np.array(detection.points.mean(axis=0))
    tracked_center = np.array(tracked_object.estimate.mean(axis=0))
    return np.linalg.norm(detection_center - tracked_center)

tracker = Tracker(
    distance_function=euclidean_distance,
    distance_threshold=distance_threshold,
    initialization_delay=initialization_delay,
)

# Re-Identification Setup

In [ ]:
# Reference gallery for re-identification
reference_gallery = {}
id_colors = defaultdict(lambda: (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255)))
id_remap = {}  # Maps tracker IDs to re-identified IDs
id_switch_log = []  # Log ID switches: (frame, sid, old_id, new_id, score)
prev_assignments = {}  # Track previous ID assignments
all_scores = []  # Store all histogram similarity scores

# Histogram Functions

In [ ]:
def compute_histogram(img):
    if img.size == 0:
        return None
    try:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None, [50, 60], [0, 180, 0, 256])
        cv2.normalize(hist, hist)
        return hist
    except Exception as e:
        print(f"Error computing histogram: {e}")
        return None

def compare_histograms(hist1, hist2):
    if hist1 is None or hist2 is None:
        return 0
    try:
        return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    except Exception as e:
        print(f"Error comparing histograms: {e}")
        return 0

# Main Processing Loop

In [ ]:
frame_num = 0
player_class_id = model.names.index('player') if 'player' in model.names else 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Create a copy to avoid visualization bugs
    frame = frame.copy()
    track_frame = frame.copy()  # For tracking-only output

    # YOLO detection
    try:
        results = model(frame, conf=0.3)[0]
    except Exception as e:
        print(f"Error in YOLO detection at frame {frame_num}: {e}")
        continue

    # Convert detections to Norfair format
    dets = []
    for box in results.boxes:
        cls_id = int(box.cls.item())
        if cls_id == player_class_id:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf.item()
            points = np.array([[x1, y1], [x2, y2]])
            dets.append(Detection(points=points, scores=np.array([conf])))

    # Filter to keep only the most confident detection (single player)
    if len(dets) > 1:
        dets = [max(dets, key=lambda x: x.scores[0])]
    print(f"Frame {frame_num}: YOLO detected {len(dets)} players")

    # Update tracker
    try:
        tracked_objects = tracker.update(detections=dets)
        print(f"Frame {frame_num}: Norfair tracks {len(tracked_objects)} objects")
    except Exception as e:
        print(f"Error in Norfair tracker at frame {frame_num}: {e}")
        continue

    # Build reference gallery (first 50 frames)
    if frame_num < num_reference_frames:
        for obj in tracked_objects:
            tid = obj.id
            if tid not in reference_gallery:
                x1, y1 = map(int, obj.estimate[0])
                x2, y2 = map(int, obj.estimate[1])
                crop = frame[y1:y2, x1:x2]
                if crop.size != 0:
                    reference_gallery[tid] = crop

    # Re-identification for later frames
    else:
        for obj in tracked_objects:
            sid = obj.id
            x1, y1 = map(int, obj.estimate[0])
            x2, y2 = map(int, obj.estimate[1])
            crop = frame[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            hist_new = compute_histogram(crop)
            best_id, best_score = None, 0

            for gallery_id, ref_crop in reference_gallery.items():
                hist_ref = compute_histogram(ref_crop)
                score = compare_histograms(hist_ref, hist_new)
                all_scores.append(score)
                if score > best_score and score > match_threshold:
                    best_score = score
                    best_id = gallery_id

            new_real_id = best_id if best_id is not None else 1  # Default to ID 1 for single player
            if sid in prev_assignments and prev_assignments[sid] != new_real_id:
                id_switch_log.append((frame_num, sid, prev_assignments[sid], new_real_id, best_score))
            prev_assignments[sid] = new_real_id
            id_remap[sid] = new_real_id

    # Draw annotations for tracking output
    for obj in tracked_objects:
        x1, y1 = map(int, obj.estimate[0])
        x2, y2 = map(int, obj.estimate[1])
        sid = obj.id
        color = id_colors[sid]

        # Bounding box and ID
        cv2.rectangle(track_frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID {sid}"
        (text_w, text_h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(track_frame, (x1, y1 - text_h - baseline - 4), (x1 + text_w, y1), (0, 0, 0), -1)
        cv2.putText(track_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Head region
        head_y2 = y1 + (y2 - y1) // 4
        cv2.rectangle(track_frame, (x1, y1), (x2, head_y2), (0, 0, 255), 1)

    # Draw annotations for re-identification output
    for obj in tracked_objects:
        x1, y1 = map(int, obj.estimate[0])
        x2, y2 = map(int, obj.estimate[1])
        sid = obj.id
        real_id = id_remap.get(sid, sid) if frame_num >= num_reference_frames else sid
        color = id_colors[real_id]

        # Bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID {real_id}"
        (text_w, text_h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(frame, (x1, y1 - text_h - baseline - 4), (x1 + text_w, y1), (0, 0, 0), -1)
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Head region
        head_y2 = y1 + (y2 - y1) // 4
        cv2.rectangle(frame, (x1, y1), (x2, head_y2), (0, 0, 255), 1)


    # Add frame info
    cv2.putText(frame, f"Frame: {frame_num}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    cv2.putText(track_frame, f"Frame: {frame_num}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
    cv2.putText(track_frame, f"FPS: {int(fps)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Write frames
    reid_out.write(frame)
    track_out.write(track_frame)

    # Save sample frame
    if frame_num % 30 == 0:
        cv2.imwrite(f'/kaggle/working/sample_frame_{frame_num}.jpg', frame)
        print(f"Frame {frame_num}: Saved sample frame")

    frame_num += 1

# Cleanup
cap.release()
reid_out.release()
track_out.release()
print(f"🎥 Tracking video saved at: {track_output_path}")
print(f"🎥 Re-ID video saved at: {reid_output_path}")

# Analysis and Visualization

In [ ]:
# Save ID switch log
switch_df = pd.DataFrame(id_switch_log, columns=["Frame", "SID", "Old_ID", "New_ID", "Score"])
switch_df.to_csv("/kaggle/working/id_switch_log.csv", index=False)
print("✅ ID switch log saved")

# Plot scores
plt.figure(figsize=(10, 5))
plt.plot(all_scores, color="blue", label="Histogram Match Score")
plt.axhline(match_threshold, color="red", linestyle="--", label=f"Threshold ({match_threshold})")
plt.title("Histogram Similarity Scores Over Time")
plt.xlabel("Comparison #")
plt.ylabel("Correlation Score")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/kaggle/working/reid_scores_plot.png")
plt.close()
print("✅ Re-ID score plot saved")